In [3]:
import pandas as pd

/tmp/ipykernel_145477/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
df = pd.read_csv('devices_v2.csv')
df.head()

,id,brand,name,type,network_technology,length,width,height,weight,resolution_l,...,sim_count,os_type,os_version,price,released_date,cpu_core,ram,internal_storage,fan_count,view_count
0,1,INFINIX,Infinix S3X,phone,4G,156.7,75.3,8.0,150.0,720.0,...,2.0,Android,8.1,NaN,2018.0,8.0,3GB,32GB,10,485
1,2,INFINIX,Infinix Note 5 Stylus,phone,4G,158.0,75.0,8.4,173.0,1080.0,...,2.0,Android,8.1,NaN,2018.0,8.0,4GB,32GB,31,524
2,3,INFINIX,Infinix Hot 6X,phone,4G,157.0,75.0,8.0,168.0,720.0,...,2.0,Android,8.1,NaN,2018.0,8.0,2GB,16GB,11,736
3,4,INFINIX,Infinix Smart 2 HD,phone,3G,156.9,76.5,8.2,165.0,720.0,...,2.0,Android,8.1,NaN,2018.0,8.0,1GB,16GB,1,335
4,5,INFINIX,Infinix Note 5,phone,4G,158.0,75.0,8.4,173.0,1080.0,...,2.0,Android,8.1,90.0,2018.0,8.0,3GB,32GB,35,871


In [8]:
df['sim_count'].value_counts()

sim_count
1.0    2709
2.0    2400
0.0     590
3.0       7
4.0       2
Name: count, dtype: int64

In [12]:
df_clean = df.dropna(subset=['price'])
df_clean.head()

,id,brand,name,type,network_technology,length,width,height,weight,resolution_l,...,sim_count,os_type,os_version,price,released_date,cpu_core,ram,internal_storage,fan_count,view_count
4,5,INFINIX,Infinix Note 5,phone,4G,158.0,75.0,8.4,173.0,1080.0,...,2.0,Android,8.1,90.0,2018.0,8.0,3GB,32GB,35,871
25,26,INFINIX,Infinix Smart 5 Pro,phone,4G,164.4,75.6,9.5,NaN,720.0,...,2.0,Android,11,70.0,2021.0,8.0,2GB,32GB,10,311
26,27,INFINIX,Infinix Hot 11 Play,phone,4G,171.8,78.0,8.9,NaN,720.0,...,2.0,Android,11,120.0,2021.0,8.0,4GB,64GB,22,1
27,28,INFINIX,Infinix Note 11i,phone,4G,173.2,78.7,8.8,NaN,1080.0,...,2.0,Android,11,140.0,2021.0,8.0,4GB,64GB,19,267
28,29,INFINIX,Infinix Note 11s,phone,4G,173.1,78.4,8.7,NaN,1080.0,...,2.0,Android,11,180.0,2021.0,8.0,4GB,64GB,44,607


### First Question

#### 1.Do mobile devices with different types of SIM cards have different prices?

##### Null Hypothesis (H0): There is no difference in the average price of mobile devices across different SIM card configurations.

##### Alternative Hypothesis (H1): There is a difference in the average price of mobile devices across different SIM card configurations.

In [14]:
df_clean['sim_count'] = df_clean['sim_count'].astype('category')
df_clean['sim_count']

/tmp/ipykernel_138548/539397305.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['sim_count'] = df_clean['sim_count'].astype('category')


4       2.0
25      2.0
26      2.0
27      2.0
28      2.0
       ... 
5667    2.0
5669    1.0
5672    1.0
5685    1.0
5686    1.0
Name: sim_count, Length: 4364, dtype: category
Categories (5, float64): [0.0, 1.0, 2.0, 3.0, 4.0]

In [21]:
from scipy.stats import f_oneway

In [22]:
groups = df_clean.groupby('sim_count')['price'].apply(list)
groups

/tmp/ipykernel_138548/2781704084.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groups = df_clean.groupby('sim_count')['price'].apply(list)


sim_count
0.0    [80.0, 100.0, 80.0, 130.0, 100.0, 110.0, 80.0,...
1.0    [250.0, 130.0, 80.0, 120.0, 130.0, 60.0, 90.0,...
2.0    [90.0, 70.0, 120.0, 140.0, 180.0, 100.0, 100.0...
3.0                     [90.0, 90.0, 100.0, 60.0, 120.0]
4.0                                               [90.0]
Name: price, dtype: object

In [23]:
anova_result = f_oneway(*groups)
print(f"F-statistic: {anova_result.statistic}, p-value: {anova_result.pvalue}")

F-statistic: 9.354801394260056, p-value: 1.586215872255128e-07


##### Assume α=0.05, as p-value indicates it is less than that which leads to the rejection of null hypothesis.

#### 2.Do mobile devices with different sizes have different prices?

##### Let's consider "weight" as a proxy for size.
##### Assuming that heavier devices might be larger or more feature-rich, potentially affecting their price.

In [25]:
df_clean = df.dropna(subset=['weight', 'price'])
df_clean.head()

,id,brand,name,type,network_technology,length,width,height,weight,resolution_l,...,sim_count,os_type,os_version,price,released_date,cpu_core,ram,internal_storage,fan_count,view_count
4,5,INFINIX,Infinix Note 5,phone,4G,158.0,75.0,8.4,173.0,1080.0,...,2.0,Android,8.1,90.00,2018.0,8.0,3GB,32GB,35,871
29,30,INFINIX,Infinix Note 11,phone,4G,164.4,76.7,7.9,184.0,1080.0,...,2.0,Android,11,100.00,2021.0,8.0,4GB,64GB,61,1
32,33,INFINIX,Infinix Hot 11s,phone,4G,168.9,77.0,8.8,205.0,1080.0,...,2.0,Android,11,917245.60,2021.0,8.0,4GB,64GB,64,1
33,34,INFINIX,Infinix Hot 11,phone,4G,164.7,76.2,8.9,201.0,1080.0,...,2.0,Android,11,110.00,2021.0,8.0,4GB,64GB,26,1
34,35,INFINIX,Infinix Zero X Pro,phone,4G,164.1,75.7,7.8,193.0,1080.0,...,2.0,Android,11,220.66,2021.0,8.0,8GB,128GB,198,2


In [26]:
from scipy.stats import linregress

In [27]:
weight = df_clean['weight']
price = df_clean['price']

slope, intercept, r_value, p_value, std_err = linregress(weight, price)
print(f"Slope (Coefficient): {slope}")
print(f"Intercept: {intercept}")
print(f"R-squared: {r_value**2}")
print(f"P-value: {p_value}")

Slope (Coefficient): 27.81554451668043
Intercept: 5131.389992042114
R-squared: 0.000944722277267647
P-value: 0.046260815693868265


##### Slope Value  
for each additional unit of weight, 
The price of the mobile device increases by about 27.82 units.This indicates a positive relationship between weight and price.
##### Intercept
No meaning! cause weight of 0 for a phone is not practically feasible in this context.
##### R-squared
Is closed to 0 so this means that weight, on its own, is not a good predictor of price for mobile devices in this dataset. There are likely other factors that have a much stronger influence on the price.
##### P-value
It's below the alpha=0.05 thus this suggests that there is a statistically significant relationship between weight and price.

### Second Question

#### Null Hypothesis (H0): There is no difference in the mean PPI density between large and small mobile devices.
#### Alternative Hypothesis (H1): There is a difference in the mean PPI density between large and small mobile devices.

In [9]:
df['size_category'] = df['screen_size'].apply(lambda x: 'large' if x >= 7.00 else 'small')

In [10]:
from scipy.stats import ttest_ind

In [16]:
large_devices = df[df['size_category'] == 'large']['ppi'].dropna()
small_devices = df[df['size_category'] == 'small']['ppi'].dropna()


In [17]:
t_stat, p_value = ttest_ind(large_devices, small_devices)
print(f"T-statistic: {t_stat}, P-value: {p_value}")


T-statistic: -7.852605710294054, P-value: 4.893430093041532e-15


##### T-statistics
Indicates significant difference between the mean PPI densities of large and small mobile devices
##### P-value
It's below the alpha=0.05 and extremely small thus this suggests that null hypothesis is not true.
The results strongly suggest that there is a statistically significant difference in PPI density between the two groups, with small devices likely having higher PPI densities compared to larger ones.

### Third Question

#### Null Hypothesis (H0): There is no difference in the mean weight of Android and iOS mobile devices.
#### Alternative Hypothesis (H1): There is a difference in the mean weight of Android and iOS mobile devices.

In [18]:
android_weights = df[df['os_type'] == 'Android']['weight'].dropna()
ios_weights = df[df['os_type'] == 'iOS']['weight'].dropna()

In [22]:
t_stat, p_value = ttest_ind(android_weights, ios_weights)
print(f"T-statistic: {t_stat}, P-value: {p_value}")

T-statistic: -8.27257493916851, P-value: 1.8867397072458762e-16


##### T-statistics
The magnitude of the T-statistic reflects a substantial difference between the two groups.
##### P-value
It's below the alpha=0.05 and extremely small thus this suggests that null hypothesis is not true.

### Fourth Question

#### Null Hypothesis (H0): The mean battery capacity per hour is the same across the different combinations of screen size classes and brands (Apple, Samsung, Xiaomi).
#### Alternative Hypothesis (H1): At least one group's mean battery capacity per hour differs from the others.

In [26]:
screen_size_threshold = 7.00
df['screen_size_class'] = df['screen_size'].apply(lambda x: 'large' if x >= screen_size_threshold else 'small')
df['screen_size_class']

0       small
1       small
2       small
3       small
4       small
        ...  
5704    small
5705    small
5706    small
5707    small
5708    small
Name: screen_size_class, Length: 5709, dtype: object

In [33]:
df_filtered = df[df['brand'].isin(['APPLE', 'SAMSUNG', 'XIAOMI'])]
df_filtered.head()

,id,brand,name,type,network_technology,length,width,height,weight,resolution_l,...,os_version,price,released_date,cpu_core,ram,internal_storage,fan_count,view_count,size_category,screen_size_class
534,535,APPLE,Apple iPad mini Wi-Fi,tablet,2G,200.0,134.7,7.2,308.0,768.0,...,6,200.0,2012.0,8.0,NaN,NaN,221,5,large,large
535,536,APPLE,Apple iPad mini Wi-Fi + Cellular,tablet,4G,200.0,134.7,7.2,312.0,768.0,...,6,300.0,2012.0,8.0,NaN,NaN,150,5,large,large
536,537,APPLE,Apple iPad 4 Wi-Fi,tablet,2G,241.2,185.7,9.4,652.0,1536.0,...,6,350.0,2012.0,8.0,1GB,16GB,162,2,large,large
537,538,APPLE,Apple iPad 4 Wi-Fi + Cellular,tablet,4G,241.2,185.7,9.4,662.0,1536.0,...,6,500.0,2012.0,8.0,1GB,16GB,180,5,large,large
538,539,APPLE,Apple iPhone 5,phone,4G,123.8,58.6,7.6,112.0,640.0,...,6,340.0,2012.0,8.0,1GB,16GB,1973,56,small,small


In [34]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [37]:
model = ols('battery_capacity ~ C(screen_size_class) * C(brand)', data=df_filtered).fit()

In [39]:
anova_results = sm.stats.anova_lm(model, typ=2)
print(anova_results)

                                     sum_sq      df            F  \
C(screen_size_class)           2.985135e+09     1.0  1380.791455   
C(brand)                       1.543471e+09     2.0   356.970718   
C(screen_size_class):C(brand)  1.106721e+08     2.0    25.596002   
Residual                       3.779004e+09  1748.0          NaN   

                                      PR(>F)  
C(screen_size_class)           3.019696e-223  
C(brand)                       1.010376e-130  
C(screen_size_class):C(brand)   1.105256e-11  
Residual                                 NaN  


### Fifth Question

#### Null Hypothesis (H0): There is no difference in the mean price of mobile devices between the different screen size classes (large vs. small).
#### Alternative Hypothesis (H1): There is a difference in the mean price of mobile devices between the different screen size classes (large vs. small).

In [42]:
model = ols('price ~ C(screen_size_class)', data=df).fit()
anova_results = sm.stats.anova_lm(model, typ=2)

print(anova_results)

                            sum_sq      df         F    PR(>F)
C(screen_size_class)  4.029644e+10     1.0  3.665762  0.055607
Residual              4.794994e+13  4362.0       NaN       NaN


The p-value of 0.055607 is just above the common significance level threshold of 0.05 so evidence is not strong enough to reject the null hypothesis.

### Sixth Question

#### Null Hypothesis (H0): The mean weight of large mobile devices is less than or equal to the mean weight of small mobile devices. 
#### Alternative Hypothesis (H1): The mean weight of large mobile devices is greater than the mean weight of small mobile devices. 

In [43]:
large_weights = df[df['size_category'] == 'large']['weight'].dropna()
small_weights = df[df['size_category'] == 'small']['weight'].dropna()

In [44]:
t_stat, p_value_two_sided = ttest_ind(large_weights, small_weights, equal_var=False)

p_value_one_sided = p_value_two_sided / 2

if t_stat > 0 and p_value_one_sided < 0.05:
    print(f"Evidence supports the alternative hypothesis with t-statistic = {t_stat}, one-sided p-value = {p_value_one_sided}")
else:
    print(f"No evidence to support the alternative hypothesis with t-statistic = {t_stat}, one-sided p-value = {p_value_one_sided}")

Evidence supports the alternative hypothesis with t-statistic = 40.88793412131005, one-sided p-value = 4.009929556855361e-159


With a p-value far below any conventional significance level (e.g., 0.05, 0.01), we can confidently reject the null hypothesis